In [2]:
#import cv2
#import cv2
import pandas as pd
import numpy as np
import time
#from pyzbar.pyzbar import decode
from datetime import datetime
import os
import shutil  # ✅ for copying files

# --- Load Initial Data ---
df = pd.read_csv("grocery_df.csv")

# Convert necessary date columns
for col in ['Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date']:
    df[col] = pd.to_datetime(df[col])

# --- Status Function ---
def get_status(row, today):
    if today < row['Discount_Date']:
        return 'Fresh'
    elif row['Discount_Date'] <= today < row['Donation_Date']:
        return 'Discount'
    elif row['Donation_Date'] <= today < row['Expiration_Date']:
        return 'Donation'
    else:
        return 'Expired'

def update_status(df, today):
    df['Status'] = df.apply(lambda row: get_status(row, today), axis=1)
    return df

# --- Table Generator ---
def generate_tables(df):
    inventory_table = df[['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name',
                          'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity', 'Unit_Price',
                          'Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date',
                          'Shelf_Life_(Days)', 'Shelf_Life_(Years)', 'Shelf_Life_Bin', 'Status']]

    fresh_table = inventory_table[df['Status'] == 'Fresh']
    discount_table = inventory_table[df['Status'] == 'Discount']
    donation_table = inventory_table[df['Status'] == 'Donation']
    expired_table = inventory_table[df['Status'] == 'Expired']

    return inventory_table, fresh_table, discount_table, donation_table, expired_table

# --- Set path of already existing backup folder ---
backup_dir = r'C:/Users/PC/WALMART/inventory/my-app/public'  # Change this to your actual path if needed

# --- Real-Time Simulation with JSON Export + Copy ---
def simulate_realtime_update(interval_sec=10, duration_sec=60):
    global df
    start_time = time.time()
    while (time.time() - start_time) < duration_sec:
        today = pd.Timestamp('2025-05-31')  # fixed date for testing
        df = update_status(df, today)
        inventory_table, fresh_table, discount_table, donation_table, expired_table = generate_tables(df)

        # Save JSON to current directory
        inventory_table.to_json("inventory.json", orient='records', date_format='iso')
        fresh_table.to_json("fresh.json", orient='records', date_format='iso')
        discount_table.to_json("discount.json", orient='records', date_format='iso')
        donation_table.to_json("donation.json", orient='records', date_format='iso')
        expired_table.to_json("expired.json", orient='records', date_format='iso')

        # Copy them to backup folder
        shutil.copy("inventory.json", os.path.join(backup_dir, "inventory.json"))
        shutil.copy("fresh.json", os.path.join(backup_dir, "fresh.json"))
        shutil.copy("discount.json", os.path.join(backup_dir, "discount.json"))
        shutil.copy("donation.json", os.path.join(backup_dir, "donation.json"))
        shutil.copy("expired.json", os.path.join(backup_dir, "expired.json"))

        print("JSON files saved and copied to backup.")
        time.sleep(interval_sec)

# --- Start Real-Time Simulation ---
simulate_realtime_update(interval_sec=5, duration_sec=20)

JSON files saved and copied to backup.
JSON files saved and copied to backup.
JSON files saved and copied to backup.
JSON files saved and copied to backup.
